# HUQ data processing

In [ ]:
import petl as etl
from utils.paths import HUQ, SITE

In [ ]:
ticketing = (
    etl
    .fromcsv(HUQ / 'mastercard.csv')
    .convertnumbers()
    .convert('datestamp', etl.dateparser('%Y-%m-%d'))
)

In [ ]:
ticketing

In [ ]:
OUTPUT = SITE / 'experiments/huq/_data'
OUTPUT.mkdir(exist_ok=True, parents=True)

In [ ]:
(
    ticketing
    .selecteq('variable', 'average_account_count')
    .cutout('variable')
    .tocsv(OUTPUT / 'footfall_proxy.csv')
)

In [ ]:
(
    ticketing
    .selecteq('variable', 'transaction_amount')
    .cutout('variable')
    .tocsv(OUTPUT / 'spend_index.csv')
)

* purchasing_power_quantile
* estimated_actual_footfall
* estimated_actual_footfall_rolling
* indexed_signal
* indexed_signal_rolling

In [ ]:
footfall = (
    etl
    .fromcsv(str(HUQ / 'footfall.long.csv.bz2'))
    .selecteq('variable', 'estimated_actual_footfall_rolling')
    .cut('datestamp', 'centre_name', 'value')
    .convert('value', int)
    .selectgt('value', 0)
    .cache()
)

In [ ]:
import re
for k, t in footfall.facet('centre_name').items():
    filename = re.sub(r'[\s-]+', '_', k.lower()) + '.csv'
    t.tocsv(OUTPUT / 'footfall' / filename)